In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
from sympy import symbols, Eq, solve
import pandas as pd

In [109]:
# --- Given Data ---
b = 300  # mm, width of section
h = 450 # mm, total height
c = 40  # mm, cover from top and bottom
d = h-c
# h = d + c  # mm, effective height of section
n = 7.51  # Modular ratio
tr = 25.4 # tension reinforcement
n_tr = 4 # no of tension reinforcement
cr = 25.4 # compression reinforcement
n_cr =2 # no of compression reinforcement
# Reinforcement details
# Asc = n_cr * (m.pi / 4) * cr**2  # Compression reinforcement area (mm^2)
# As = n_tr * (m.pi / 4) * tr**2  # Tension reinforcement area (mm^2)
As = 3039 # mm^2, tension reinforcement area (mm^2)
Asc = 1548 # mm^2, compression reinforcement area (mm^2)
# Material properties
f_ck = 25  # MPa, characteristic compressive strength of concrete
f_y =  276 # MPa, yield strength of steel
gamma_c = 1.5  # Safety factor for concrete
gamma_s = 1.15  # Safety factor for steel
alpha_cc = 0.85  # Concrete strength reduction factor
Es = 200000  # Young's modulus of steel

# Design strengths
fcd = alpha_cc * (f_ck / gamma_c)  # MPa
# fcd = 20.7  # MPa
fyd = f_y / gamma_s  # MPa


f_cm = f_ck + 8  # MPa, mean compressive strength of concrete
# Calculate modulus of elasticity of concrete (Ec) in MPa
Ec = 22 * ((f_cm / 10) ** 0.3) * 1000  # Convert GPa to MPa
print(f"Modulus of Elasticity of Concrete (Ec): {Ec} MPa")
Ct = 1.361
n = round(Es / Ec * ( 1 + Ct ),2) # Modular ratio
print("Modular ratio (n):", n)


Modulus of Elasticity of Concrete (Ec): 31475.806210019346 MPa
Modular ratio (n): 15.0


In [3]:
def newtonR(f, x0):
    tolf = 1e-08
    tolx = 1e-08
    dx = 1e-08
    df = lambda x: (f(x + dx) - f(x)) / dx
    iter_root = [x0]
    for k in range(1000):
        iter_root.append(iter_root[k] - (f(iter_root[k]) / df(iter_root[k])))
        if abs(f(iter_root[k + 1])) <= tolf or abs(iter_root[k + 1] - iter_root[k]) <= tolx:
            return iter_root[k + 1]
    raise ValueError("Newton-Raphson method did not converge.")

In [4]:
# --- Calculations ---
# Ultimate Neutral axis depth (x) from force equilibrium
x_ult = (As - Asc) * fyd / (0.8 * fcd * b)

# Design moment resistance (Mrd)
M_rd = As * fyd * (d - 0.4 * x_ult) / (10**6)  # Convert to kNm
M_rd2 = (fcd * 0.8 * x_ult * b * (d - 0.4 * x_ult) + fyd * Asc * (d - c)) * 1e-6

# Uncracked moment of inertia (I_uncr) - for a rectangular section
Eqn_NA_Uncracked = (
    lambda x_uncr: ((b * x_uncr**2) / 2)
    + (n * Asc * (x_uncr - c))
    - ((b * (h - x_uncr) ** 2) / 2)
    - (n * As * (d - x_uncr))
)
NA_uncracked = newtonR(Eqn_NA_Uncracked, 100)  # Neutral axis of uncracked section
# print("NA_uncracked:",NA_uncracked,"mm")

I_uncr = (
    (b / 3) * NA_uncracked**3
    + n * Asc * (NA_uncracked - c) ** 2
    + (b / 3) * (h - NA_uncracked) ** 3
    + n * As * (d - NA_uncracked) ** 2
)
I_uncr_in_inches = I_uncr / (25.4**4)  # Convert mm^4 to in^4
print("I_uncr:", I_uncr, "mm^4")
print("I_uncr:", I_uncr_in_inches, "in^4")


# Cracked moment of inertia (I_cr) - transformed section
Eqn_NA_cracked = lambda x_cr: (b * x_cr**2 / 2) + (n * Asc * (x_cr - c)) - (n * As * (d - x_cr))
NA_cracked = newtonR(Eqn_NA_cracked, 100)

# Parallel axis theorem to find I_cr
I_cr = (
    (b / 3) * NA_cracked**3
    + n * Asc * (NA_cracked - c) ** 2
    + n * As * (d - NA_cracked) ** 2
)  # (b * x**3) / 3 + (n * As * (d - y_n)**2) + (n * Asc * (y_n - c)**2)

# First moment of cracking (M_cr)
f_ctm = 0.3 * (f_ck ** (2 / 3))  # MPa, mean tensile strength of concrete
y = h / 2  # Distance to extreme fiber
f_ctm_psi = f_ctm * 145.038  # Convert MPa to psi
print("f_ctm:", f_ctm, "MPa")
print("f_ctm:", f_ctm_psi, "psi")
M_cr = (f_ctm * I_uncr) / (h - NA_uncracked) / (10**6)  # Convert to kNm

results = {
    "Neutral Axis (Uncracked)": (f"{NA_uncracked:.3f} mm", "((b * x_uncr^2) / 2)+(n * Asc * (x_uncr - c)) = ((b * (h - x_uncr)^2) / 2)+(n * As * (d - x_uncr))"),
    "Moment of Inertia (Uncracked)": (I_uncr, "(b / 3) * NA_uncracked^3 + n * Asc * (NA_uncracked - c)^2 + (b / 3) * (h - NA_uncracked)^3 + n * As * (d - NA_uncracked)^2"),
    "Neutral Axis (Cracked)": (f"{NA_cracked:.3f} mm", "(b * x_cr^2 / 2) + (n * Asc * (x_cr - c)) = (n * As * (d - x_cr))"),
    "Moment of Inertia (Cracked)": (I_cr, "(b / 3) * NA_cracked^3 + n * Asc * (NA_cracked - c)^2 + n * As * (d - NA_cracked)^2"),
    "Neutral Axis Depth at failure (x_ult)": (f"{x_ult:.3f} mm", "(As - Asc) * fyd / (0.8 * fcd * b)"),
    "Moment of Resistance M_rd": (M_rd, "M_rd = As * fyd * (d - 0.4 * x)"),
    # "Moment of Resistance M_rd2": (M_rd2, "M_rd2 = (fcd * 0.8 * x * b * (d - 0.4 * x) + fyd * Asc * (d - c)) * 1e-6"),
    "Moment of First Crack (M_cr)": (M_cr, "M_cr = (f_ctm * I_uncr) / (h - x)"),
}

print("--- Results ---")
for key, value in results.items():
    print(f"{key}: {value[0]}")

I_uncr: 4548978612.055641 mm^4
I_uncr: 10928.96483127929 in^4
f_ctm: 2.564963920015045 MPa
f_ctm: 372.01723703114214 psi
--- Results ---
Neutral Axis (Uncracked): 245.301 mm
Moment of Inertia (Uncracked): 4548978612.055641
Neutral Axis (Cracked): 198.893 mm
Moment of Inertia (Cracked): 3404573045.379113
Neutral Axis Depth at failure (x_ult): 105.247 mm
Moment of Resistance M_rd: 268.3324020705883
Moment of First Crack (M_cr): 57.000709833202315


In [5]:
ρ = As / (b * d)  # Reinforcement ratio
ρ_cr = Asc / (b * d)  # Compression reinforcement ratio

k = ((((ρ+ρ_cr)**2 * n**2) + (2*n*(ρ + ρ_cr*(c/d))))** 0.5) - ((ρ + ρ_cr) * n) # Neutral axis depth ratio
print("neutral axis depth (mm):", k * d)
print("neutral axis depth (inches):", (k * d) / 25.4)

moment = 100  # kN·m

from sympy import symbols, Eq, solve
import pandas as pd
fc = symbols('fc')
fsc = ((k*d - c) / (k * d)) * (n*fc)
fs = ((1-k)/k) *n*fc
eq = Eq(0.5 * fc * b * k * d * (d - (k*d/3)) + (fsc * Asc * (d - c)), moment*1e6)
sol = solve(eq, fc)
print("fc:", sol[0], "MPa")
print("fs:", fs.subs(fc, sol[0]), "MPa")
print("fsc:", fsc.subs(fc, sol[0]), "MPa")

neutral axis depth (mm): 198.89341501066895
neutral axis depth (inches): 7.830449409868857
fc: 5.84194882470267 MPa
fs: 93.0101581793894 MPa
fsc: 70.0058771949372 MPa


In [16]:
import openpyxl
# Generate a range of moments from 0 to M_cr
moments = np.linspace(0, M_cr, 100)
fc_values = []
fs_values = []
fsc_values = []

# Create a new workbook and select the active worksheet
wb = openpyxl.Workbook()
ws = wb.active

# Write headers
ws.append(['Moment (kNm)', 'fc (MPa)', 'fs (MPa)', 'fsc (MPa)'])

# Populate fc_values, fs_values, and fsc_values by solving for each moment
for moment in moments:
    eq = Eq(0.5 * fc * b * k * d * (d - (k * d / 3)) + (fsc * Asc * (d - c)), moment * 1e6)
    sol = solve(eq, fc)
    if sol:
        fc_value = float(sol[0])
        fc_values.append(fc_value)
        fs_values.append(float(fs.subs(fc, fc_value)))
        fsc_values.append(float(fsc.subs(fc, fc_value)))
    else:
        fc_values.append(0)
        fs_values.append(0)
        fsc_values.append(0)

# Write data to the worksheet
for moment, fc_value, fs_value, fsc_value in zip(moments, fc_values, fs_values, fsc_values):
    ws.append([moment, fc_value, fs_value, fsc_value])

# Save the workbook to an Excel file
wb.save('moments_fc_fs_fsc_values.xlsx')


In [49]:
fc_values[-1]/fs_values[-1] * d / 1.062 # Ratio of fc to fs at first crack

24.248601993974244

In [108]:
strain_concrete = []
strain_steel = []
strain_steel_compression = []
for i in range(len(fc_values)):
    strain_concrete.append(fc_values[i] / Ec)  # Strain in concrete
    strain_steel.append(fs_values[i] / Es)  # Strain in steel
    strain_steel_compression.append(fsc_values[i] / Es)  # Strain in compression steel  
print(strain_concrete)
strain_concrete[-9]/strain_steel[-9] #* d / (1+(strain_concrete[-9]/strain_steel[-9]))# Ratio of strain in concrete to strain in steel at first crack
 

[0.0, 1.06862653732503e-06, 2.137253074650053e-06, 3.205879611975098e-06, 4.274506149300125e-06, 5.343132686625151e-06, 6.411759223950177e-06, 7.480385761275222e-06, 8.54901229860025e-06, 9.617638835925275e-06, 1.0686265373250302e-05, 1.1754891910575347e-05, 1.2823518447900373e-05, 1.3892144985225401e-05, 1.4960771522550427e-05, 1.6029398059875472e-05, 1.70980245972005e-05, 1.8166651134525525e-05, 1.923527767185055e-05, 2.030390420917554e-05, 2.137253074650053e-05, 2.2441157283825708e-05, 2.3509783821150693e-05, 2.4578410358475682e-05, 2.564703689580067e-05, 2.6715663433125847e-05, 2.7784289970450836e-05, 2.8852916507775828e-05, 2.9921543045100814e-05, 3.0990169582425806e-05, 3.205879611975098e-05, 3.312742265707597e-05, 3.4196049194400956e-05, 3.526467573172595e-05, 3.6333302269051124e-05, 3.740192880637611e-05, 3.84705553437011e-05, 3.953918188102609e-05, 4.060780841835108e-05, 4.167643495567626e-05, 4.274506149300125e-05, 4.381368803032624e-05, 4.488231456765122e-05, 4.5950941104976

0.39909887002398053

In [45]:
from sympy import symbols

# Define x as a symbolic variable
x = symbols('x')
neutral_axis_depths = []

for i in range(1, len(fc_values)):  # Start range from 1
    fc_value = fc_values[i]
    fs_value = fs_values[i]
    # Solve for the neutral axis depth using the equation
    neutral_axis = (((fc_value / fs_value) * d) / (1 + (fc_value / fs_value)))
    neutral_axis_depths.append(neutral_axis)

# Print the calculated neutral axis depths
print("Neutral Axis Depths:", neutral_axis_depths)


Neutral Axis Depths: [24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044, 24.230126101702044